In [1]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
import pygame

2024-07-08 14:49:12.369103: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 14:49:12.372118: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-08 14:49:12.383350: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-08 14:49:12.404568: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-08 14:49:12.404615: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-08 14:49:12.419183: I tensorflow/core/platform/cpu_feature_guard.cc:

pygame 2.6.0 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
model = tf.keras.models.load_model('get_gesture_model/gesture_model.h5')
model.summary()

2024-07-08 14:49:13.989011: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-08 14:49:13.989581: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2251] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/media/benjamin/benjamin_external_ssd1/vg_motion_control/env_vg_motion_control/lib/python3.11/site-packages/keras/src/optimizers/base_optimizer.py:33: UserWarning: Argument `decay` is no longer supported and will be ignored.
  warnings.warn(


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_1 (Conv2D)               │ (None, 100, 100, 8)    │           136 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 24, 24, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 24, 24, 16)     │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 12, 12, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2304)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 512)            │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │        65,664 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │         1,290 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,247,780 (4.76 MB)

 Trainable params: 1,247,778 (4.76 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [3]:
# Initialize MediaPipe's Hands model
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=1, min_detection_confidence=0.5)

# Gesture labels
gesture_labels = {
	0: 'Left',
	1: 'Right',
}

I0000 00:00:1720442988.200739   30244 gl_context_egl.cc:85] Successfully initialized EGL. Major : 1 Minor: 5
I0000 00:00:1720442988.283327   30573 gl_context.cc:357] GL version: 3.2 (OpenGL ES 3.2 NVIDIA 535.183.01), renderer: NVIDIA GeForce RTX 2060/PCIe/SSE2


INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1720442988.295103   30560 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1720442988.317917   30552 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


In [4]:
# Initialize Pygame
pygame.init()
window_width = 800
window_height = 600
window = pygame.display.set_mode((window_width, window_height))
pygame.display.set_caption("Gesture-Based Game")

# Set up the player
player_width = 50
player_height = 50
player_x = window_width // 2 - player_width // 2
player_y = window_height - player_height - 10
player_speed = 5

# Set up the obstacle
obstacle_width = 50
obstacle_height = 50
obstacle_x = np.random.randint(0, window_width - obstacle_width)
obstacle_y = 0
obstacle_speed = 3

# Game over flag and text
game_over = False
game_over_text = pygame.font.Font('freesansbold.ttf', 64).render('Game Over', True, (255, 255, 255))

# Restart button
restart_button_text = pygame.font.Font('freesansbold.ttf', 32).render('Restart', True, (255, 255, 255))
restart_button_rect = restart_button_text.get_rect()
restart_button_rect.center = (window_width // 2, window_height // 2 + 50)

# Game loop
running = True
clock = pygame.time.Clock()

In [5]:
# Capturing video from webcam:
cap = cv2.VideoCapture(0)
while running:
	for event in pygame.event.get():
		if event.type == pygame.QUIT:
			running = False
		if event.type == pygame.MOUSEBUTTONDOWN and game_over:
			mouse_pos = pygame.mouse.get_pos()
			if restart_button_rect.collidepoint(mouse_pos):
				# Restart the game
				game_over = False
				player_x = window_width // 2 - player_width // 2
				obstacle_x = np.random.randint(0, window_width - obstacle_width)
				obstacle_y = 0

	success, frame = cap.read()
	if not success:
		break

	# Flip the frame horizontally for a mirror effect
	frame = cv2.flip(frame, 1)

# Detecting hand landmarks and recognizing gestures
	results = hands.process(frame)
	if results.multi_hand_landmarks:
		for hand_landmarks in results.multi_hand_landmarks:
			# Get the index finger landmarks
			index_finger_landmarks = hand_landmarks.landmark[mp_hands.HandLandmark.INDEX_FINGER_TIP]
			index_finger_x = int(index_finger_landmarks.x * frame.shape[1])
			index_finger_y = int(index_finger_landmarks.y * frame.shape[0])

			# Recognize gesture based on index finger position
			if index_finger_x < frame.shape[1] // 3:
				gesture = 'Left'
				player_x -= player_speed
			elif index_finger_x > frame.shape[1] * 2 // 3:
				gesture = 'Right'
				player_x += player_speed
			else:
				gesture = 'Neutral'

			# Draw a circle at the index finger position
			cv2.circle(frame, (index_finger_x, index_finger_y), 10, (0, 255, 0), -1)

			# Draw the recognized gesture on the frame
			cv2.putText(frame, gesture, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

	cv2.imshow('Gesture Recognition', frame)
	if cv2.waitKey(1) & 0xFF == ord('q'):
		break
# Updating the player's position
	if not game_over:
		# Update the player's position
		if player_x < 0:
			player_x = 0
		elif player_x > window_width - player_width:
			player_x = window_width - player_width

		# Update the obstacle's position
		obstacle_y += obstacle_speed

		# Check for collision between the player and the obstacle
		if player_x < obstacle_x + obstacle_width and \
				player_x + player_width > obstacle_x and \
				player_y < obstacle_y + obstacle_height and \
				player_y + player_height > obstacle_y:
			# Game over
			game_over = True

		# Reset the obstacle if it goes off the screen
		if obstacle_y > window_height:
			obstacle_x = np.random.randint(0, window_width - obstacle_width)
			obstacle_y = 0

		# Clear the window
		window.fill((0, 0, 0))

# Creating player and obstacle:
		pygame.draw.rect(window, (255, 255, 255), (player_x, player_y, player_width, player_height))

		# Draw the obstacle
		pygame.draw.rect(window, (255, 0, 0), (obstacle_x, obstacle_y, obstacle_width, obstacle_height))

	# Display "Game Over" text if game over
	if game_over:
		window.blit(game_over_text, 
					(window_width // 2 - game_over_text.get_width() // 2,
					window_height // 2 - game_over_text.get_height() // 2))
		pygame.draw.rect(window, (0, 0, 255), restart_button_rect)
		window.blit(restart_button_text, restart_button_rect)

	# Update the display
	pygame.display.update()

	# Limit the frame rate
	clock.tick(60)

cap.release()
cv2.destroyAllWindows()
pygame.quit()

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread (0xb5413b0).
Cannot move to target thread (0xa35ff60)

QObject::moveToThread: Current thread (0xa35ff60) is not the object's thread